In [93]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import BaggingClassifier

In [94]:
df = pd.read_csv("card_credit_fraud.csv")
df['type'] = df['type'].replace(["CASH_OUT","PAYMENT","TRANSFER","DEBIT","CASH_IN"], [0, 1, 2, 3, 4])
df.head()

,transactionId,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,0,1,1,983964,C1231006815,170136,16029636,M1979787155,0,0,0
1,1,1,1,186428,C1666544295,21249,1938472,M2044282225,0,0,0
2,2,1,2,181,C1305486145,181,0,C553264065,0,0,1
3,3,1,0,181,C840083671,181,0,C38997010,21182,0,1
4,4,1,1,1166814,C2048537720,41554,2988586,M1230701703,0,0,0


In [95]:
#separation apprentisage/test

y = df['isFraud']
#X = df.drop(["isFraud", "nameOrig", "nameDest"], axis="columns",inplace=True)
X = df.drop(["isFraud", "nameOrig", "nameDest"], axis=1)
print(X)

         transactionId  step  type     amount  oldbalanceOrg  newbalanceOrig  \
0                    0     1     1     983964         170136        16029636   
1                    1     1     1     186428          21249         1938472   
2                    2     1     2        181            181               0   
3                    3     1     0        181            181               0   
4                    4     1     1    1166814          41554         2988586   
...                ...   ...   ...        ...            ...             ...   
1008208        6362615   743     0   33968213       33968213               0   
1008209        6362616   743     2  631140928      631140928               0   
1008210        6362617   743     0  631140928      631140928               0   
1008211        6362618   743     2   85000252       85000252               0   
1008212        6362619   743     0   85000252       85000252               0   

         oldbalanceDest  newbalanceDest

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [97]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train,y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [98]:
y_pred = clf.predict(X_test)
print(y_pred) # modèle qui a prédit 

[0 0 0 ... 0 0 0]


In [99]:
#Accurancy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.9995139917477782

In [100]:
#Matrice de confusion
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

print(confusion_matrix(y_test,y_pred))

[[300099      0]
 [   147   2218]]


In [112]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    300099
           1       1.00      0.94      0.97      2365

    accuracy                           1.00    302464
   macro avg       1.00      0.97      0.98    302464
weighted avg       1.00      1.00      1.00    302464



In [103]:
from imblearn.over_sampling import SMOTE
seed = 5
smote = SMOTE(sampling_strategy='auto', random_state=seed, k_neighbors=7)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [104]:
clf.fit(X_train_smote,y_train_smote)

RandomForestClassifier(max_depth=2, random_state=0)

In [108]:
y_pred_smote = clf.predict(X_test)
print(y_pred_smote) # modèle qui a prédit 

[0 0 0 ... 0 0 0]


In [109]:
#Accurancy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred_smote)
accuracy

0.9995139917477782

In [110]:
print(confusion_matrix(y_test,y_pred_smote))

[[300099      0]
 [   147   2218]]


In [111]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_smote))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    300099
           1       1.00      0.94      0.97      2365

    accuracy                           1.00    302464
   macro avg       1.00      0.97      0.98    302464
weighted avg       1.00      1.00      1.00    302464

